In [2]:
from aeon.datasets import load_classification
X, y = load_classification("ArticularyWordRecognition")
print(" Shape of X = ", X.shape)

 Shape of X =  (575, 9, 144)


In [3]:
X_train, y_train = load_classification("ArticularyWordRecognition",  split="train")
X_test, y_test = load_classification("ArticularyWordRecognition",  split="test")

In [11]:
import os
import numpy as np
import pandas as pd
from utils.tools import create_directory

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


def fit_classifier(all_labels, X_train, y_train, X_val=None, y_val=None, epochs=10, batch_size=16):
    nb_classes = len(np.unique(all_labels))
    # Create Classifier --------------------------------------------------------
    if classifier_name == "FCN" or classifier_name == "ResNet":
        input_shape = (X_train.shape[1], X_train.shape[2])
    elif classifier_name == "lstm_dcnn" or classifier_name == "MLSTM_FCN":
        input_shape = (X_train.shape[1], X_train.shape[2])
    else:
        input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
    # Call Classifier ----------------------------------------------------------
    classifier = create_classifier(classifier_name, input_shape, nb_classes, verbose=True)
    # Train Classifier ----------------------------------------------------------
    if X_val is None:
        classifier.fit(X_train, y_train, None, None, epochs, batch_size)
    else:
        classifier.fit(X_train, y_train, X_val, y_val, epochs, batch_size)
    return classifier


def create_classifier(classifier_name, input_shape, nb_classes, verbose=False):

    # Networks ------------------------------------------------------------------------------
    if classifier_name == "T_CNN":
        from classifiers import T_CNN
        return T_CNN.Classifier_T_CNN(sub_output_directory, input_shape, nb_classes, verbose)
    if classifier_name == "S_CNN":
        from classifiers import S_CNN
        return S_CNN.Classifier_S_CNN(sub_output_directory, input_shape, nb_classes, verbose)
    if classifier_name == "ST_CNN":
        from classifiers import ST_CNN
        return ST_CNN.Classifier_ST_CNN(sub_output_directory, input_shape, nb_classes, verbose)

    if classifier_name == "DCNN_2L":
        from classifiers import DCNN_2L
        return DCNN_2L.Classifier_DCNN_2L(sub_output_directory, input_shape, nb_classes, verbose)
    if classifier_name == "DCNN_3L":
        from classifiers import DCNN_3L
        return DCNN_3L.Classifier_DCNN_3L(sub_output_directory, input_shape, nb_classes, verbose)
    if classifier_name == "DCNN_4L":
        from classifiers import DCNN_4L
        return DCNN_4L.Classifier_DCNN_4L(sub_output_directory, input_shape, nb_classes, verbose)
    
    # ------------------------------------------------------------------------------------------------------------------
    # Component Analysis -----------------------------------------------------------------------------------------------
    if classifier_name == "FCN":
        from classifiers import FCN
        return FCN.Classifier_FCN(sub_output_directory, input_shape, nb_classes, verbose)
    if classifier_name == "D_FCN":
        from classifiers import D_FCN
        return D_FCN.Classifier_D_FCN(sub_output_directory, input_shape, nb_classes, verbose)

    if classifier_name == "ResNet":
        from classifiers import ResNet
        return ResNet.Classifier_ResNet(sub_output_directory, input_shape, nb_classes, verbose)
    if classifier_name == "D_ResNet":
        from classifiers import D_ResNet
        return D_ResNet.Classifier_D_ResNet(sub_output_directory, input_shape, nb_classes, verbose)

    if classifier_name == "MC_CNN":
        from classifiers import MC_CNN
        return MC_CNN.Classifier_MC_CNN(sub_output_directory, input_shape, nb_classes, verbose)
    if classifier_name == "MLSTM_FCN":
        from classifiers import MLSTM_FCN
        return MLSTM_FCN.Classifier_MLSTM_FCN(sub_output_directory, input_shape, nb_classes, verbose)
    elif classifier_name == "lstm_dcnn":
        from classifiers import lstm_dcnn
        return lstm_dcnn.Classifier_LSTM_DCNN(sub_output_directory, input_shape, nb_classes, verbose)


def s_length(train_df, test_df):
    train_lengths = train_df.applymap(lambda x: len(x)).values
    test_lengths = test_df.applymap(lambda x: len(x)).values

    train_vert_diffs = np.abs(train_lengths - np.expand_dims(train_lengths[0, :], 0))

    if np.sum(train_vert_diffs) > 0:  # if any column (dimension) has varying length across samples
        train_max_seq_len = int(np.max(train_lengths[:, 0]))
        test_max_seq_len = int(np.max(test_lengths[:, 0]))
        max_seq_len = np.max([train_max_seq_len, test_max_seq_len])
    else:
        max_seq_len = train_lengths[0, 0]
    return max_seq_len
# Problem Setting -----------------------------------------------------------------------------------------------------
ALL_Results = pd.DataFrame()
ALL_Results_list = []
problem_index = 0
data_path = os.getcwd() + '/multivariate_ts/'
# Hyper-Parameter Setting ----------------------------------------------------------------------------------------------
classifier_name = "DCNN_2L"  # Choose the classifier name from aforementioned List
epochs = 500
Resample = 1  # Set to '1' for default Train and Test Sets, and '30' for running on all resampling
# ----------------------------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------------

for problem in os.listdir(data_path):
    # Load Data --------------------------------------------------------------------------------------------------------
    output_directory = os.getcwd() + '/Results_'
    output_directory = output_directory + classifier_name + '/' + problem + '/'
    create_directory(output_directory)
    print("[Main] Problem: {}".format(problem))
    itr_result = [problem]
    # load --------------------------------------------------------------------------
    # set data folder
    X_train, Y_train = load_classification("ArticularyWordRecognition",  split="train")
    X_test, Y_test = load_classification("ArticularyWordRecognition",  split="test")

    all_data = np.vstack((X_train, X_test))
    all_labels = np.hstack((Y_train, Y_test))
    all_indices = np.arange(len(all_data))

    sub_output_directory = output_directory + str(1) + '/'
    create_directory(sub_output_directory)
    # Default Train and Test Set
    x_train = X_train
    x_test = X_test
    y_train = Y_train
    y_test = Y_test
    
    # Making Consistent with Keras Output -------------------------------------------------
    all_labels_new = np.concatenate((y_train, y_test), axis=0)
    print("[Main] All labels: {}".format(np.unique(all_labels_new)))
    tmp = pd.get_dummies(all_labels_new).values
    y_train = tmp[:len(y_train)]
    y_test = tmp[len(y_train):]

    # Making Consistent with Keras Input ---------------------------------------------------
    if classifier_name == "FCN" or classifier_name == "ResNet" or classifier_name == "MLSTM_FCN":
        x_train = x_train.reshape(x_train.shape[0], x_train.shape[2], x_train.shape[1])
        x_test = x_test.reshape(x_test.shape[0], x_test.shape[2], x_test.shape[1])
    else:
        x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
        x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

    # classifier-----------------------------------------------------------------
    # Dynamic Batch-size base on Data
    if problem == 'EigenWorms' or problem == 'DuckDuck':
        batch_size = 1
    else:
        # batch_size = np.ceil(x_train.shape[0] / (8 * (np.max(y_train.shape[1]) + 1)))
        batch_size = 8
        
    val_index = np.random.randint(0, np.int(x_train.shape[0]), np.int(x_train.shape[0] / 10), dtype=int)
    x_val = x_train[val_index, :]
    y_val = y_train[val_index, :]

    classifier = fit_classifier(all_labels_new, x_train, y_train, x_val, y_val, epochs, batch_size)
    metrics_test, conf_mat = classifier.predict(x_test, y_test, best=True)
    metrics_test2, conf_mat2 = classifier.predict(x_test, y_test, best=False)

    metrics_test['train/val/test/test2'] = 'test'
    metrics_test2['train/val/test/test2'] = 'test2'
    metrics = pd.concat([metrics_test, metrics_test2]).reset_index(drop=True)

    print("[Main] Problem: {}".format(problem))
    print(metrics.head())

    metrics.to_csv(sub_output_directory + 'classification_metrics.csv')
    np.savetxt(sub_output_directory + 'confusion_matrix.csv', conf_mat, delimiter=",")
    itr_result.append(metrics.accuracy[0])
    itr_result.append(metrics.accuracy[1])
    sub_output_directory = []

    if len(ALL_Results_list) == 0:
        ALL_Results_list = np.hstack((ALL_Results_list, itr_result))
    else:
        ALL_Results_list = np.vstack((ALL_Results_list, itr_result))

    problem_index = problem_index + 1

ALL_Results = pd.DataFrame(ALL_Results_list)
ALL_Results.to_csv(os.getcwd() + '/Results_' + classifier_name + '/'+'All_results1.csv')


[Main] Problem: NATOPS
[Main] All labels: ['1.0' '10.0' '11.0' '12.0' '13.0' '14.0' '15.0' '16.0' '17.0' '18.0'
 '19.0' '2.0' '20.0' '21.0' '22.0' '23.0' '24.0' '25.0' '3.0' '4.0' '5.0'
 '6.0' '7.0' '8.0' '9.0']


/tmp/ipykernel_985966/3900600103.py:153: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  val_index = np.random.randint(0, np.int(x_train.shape[0]), np.int(x_train.shape[0] / 10), dtype=int)
2024-03-18 11:22:55.716269: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 11:22:56.658420: W tensorflow/compiler/tf

Creating DisJoint DCNN_2L Classifier


2024-03-18 11:22:57.472369: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-18 11:22:57.511458: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-18 11:22:57.511731: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "functional_1"

...

[DCNN_2L] Training DCNN_2L Classifier
Epoch 1/500


I0000 00:00:1710757379.727096  987614 service.cc:145] XLA service 0x7f36f8005000 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1710757379.727119  987614 service.cc:153]   StreamExecutor device (0): Quadro P400, Compute Capability 6.1
2024-03-18 11:22:59.810803: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-18 11:23:00.089693: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


15/35 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1861 - loss: 12.5642

I0000 00:00:1710757385.063903  987614 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


35/35 ━━━━━━━━━━━━━━━━━━━━ 13s 156ms/step - accuracy: 0.3204 - loss: 10.6618 - val_accuracy: 0.7037 - val_loss: 0.7593 - learning_rate: 0.0010
Epoch 2/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8532 - loss: 3.6723 - val_accuracy: 0.9630 - val_loss: 0.0739 - learning_rate: 0.0010
Epoch 3/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9846 - loss: 1.9103 - val_accuracy: 0.9630 - val_loss: 0.0559 - learning_rate: 0.0010
Epoch 4/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9961 - loss: 0.9084 - val_accuracy: 1.0000 - val_loss: 0.0288 - learning_rate: 0.0010
Epoch 5/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 1.0000 - loss: 0.5276 - val_accuracy: 1.0000 - val_loss: 0.0259 - learning_rate: 0.0010
Epoch 6/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 1.0000 - loss: 0.2981 - val_accuracy: 1.0000 - val_loss: 0.0173 - learning_rate: 0.0010
Epoch 7/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 1.0000 - loss: 0.2688 - val_accur